<h4>Created by Jaswinder Singh</h4>
<h1><center>💸G-Research💸 Indicators 🔧plug and play🔧 features !</center></h1>


## Update 
- [Volume-Moving-Average-Convergence-Divergence](#Volume-Moving-Average-Convergence-Divergence)
- [Typical Value Moving Average Convergence Divergence](#Typical-Value-Moving-Average-Convergence-Divergence)    
- [Typical Value Volume Weighted Moving Average](#Typical-Value-Volume-Weighted-Moving-Average)
- [Typical Value](#Typical-Value)
- [Percentage Value Oscillator](#Percentage-Value-Oscillator)
- [Kaufman Efficiency](#Kaufman-Efficiency)


<hr >

# <span class="title-section w3-xxlarge" id="outline">Table Of Content 📑</span>


1. [IMPORTS](#IMPORTS) 

2. [Installing NitroFe](#Installing-NitroFe) 

3. [Creating Features](#Creating-Features)
    
    3.1 [Series Weighted Moving Feature](#Series-Weighted-Moving-Feature)
    
    3.2 [Percentage Value Oscillator](#Percentage-Value-Oscillator)

    3.3 [Moving Average Convergence Divergence](#Moving-Average-Convergence-Divergence)
    
    3.4 [Typical Value](#Typical-Value)
    
    3.5 [Kaufman Efficiency](#Kaufman-Efficiency)
    
    3.6 [Average Directional Movement Index](#Average-Directional-Movement-Index)
    
    3.7 [Moving average Features](#Moving-average-Features)
    
    3.8 [Relative Strength Index](#Relative-Strength-Index)
    
    3.9 [Inverse Fisher Relative Strength Index](#Inverse-Fisher-Relative-Strength-Index)
    
    3.10 [Aroon Oscillator](#Aroon-Oscillator)
    
    3.11 [Triple Exponential Moving Average Oscillator](#Triple-Exponential-Moving-Average-Oscillator)
    
4. [Combination Features](#Combination-Features)

    4.1 [Typical Value Moving Average Convergence Divergence](#Typical-Value-Moving-Average-Convergence-Divergence)
    
    4.2 [Typical Value Volume Weighted Moving Average](#Typical-Value-Volume-Weighted-Moving-Average)
    
    4.3 [Volume-Moving-Average-Convergence-Divergence](#Volume-Moving-Average-Convergence-Divergence)


<hr >

<span id="IMPORTS"><h1><center>IMPORTS</center></h1></span>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

<hr >


<span id="Installing-NitroFe"><h1><center>Installing NitroFe</center></h1></span>

In [ ]:
!pip install NitroFe

📚 To read more about NitroFe you can visit the documentation page https://github.com/NITRO-AI/NitroFE.



🌟 Like this Project? Give us a star on GitHub
<hr >

<span id="Preparing-data"><h1><center>Preparing data</center></h1></span>

In [ ]:
data = pd.read_csv('../input/g-research-crypto-forecasting/train.csv') 
asset_details = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv') 

In [ ]:
btc = data[data["Asset_ID"]==1].reset_index(drop = True)
btc['timestamp'] = pd.to_datetime(btc['timestamp'], unit='s')
btc = btc.set_index('timestamp')[-500:]

 ## Philosophy
 The goal here is to provide you basic-elements/lego pieces, such that you can stack them to create your own features ! 
 
 
 You can create a volume weighted moving average out of typical value  or create MACD out of a hull moving average, so on and so forth !

<hr >


<span id="Creating-Features"><h1><center>Creating Features</center></h1></span>

<span id="Series-Weighted-Moving-Feature"><h2><center>Series Weighted Moving Feature</center></h2></span>

$$ Series \ Weighted \ Moving \ Feature[t] = \frac{\operatorname{operation} (dataframe[(t-'lookback \ period') \to t] * dataframe \ for \ weight[(t-'lookback \ period') \to t])}{\operatorname{operation} (dataframe \ for \ weight[(t-'lookback \ period') \to t])} $$

In [ ]:
from NitroFE import SeriesWeightedMovingFeature
ob = SeriesWeightedMovingFeature(lookback_period=8, operation= np.mean)
res = ob.fit(btc['Close'][-500:],btc['Volume'][-500:],first_fit=True)['Close'][-100:]

from NitroFE import ExponentialMovingFeature
ob = ExponentialMovingFeature(span=8)
res2 = ob.fit(btc['Close'][-500:], first_fit=True)[-100:]

In [ ]:
data=btc[-100:]
candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                           open = data[('Open')], 
                                           high = data[('High')], 
                                           low = data[('Low')], 
                                           close = data[('Close')])])

candlestick.add_trace(go.Scatter(x=data.index, y=res.values,mode='lines',line=dict(color='royalblue', width=5),name='Volume Weighted Closing Moving Feature'))
candlestick.add_trace(go.Scatter(x=data.index, y=res2.values,mode='lines',line=dict(color='yellow', width=5),name='Exponential Moving Feature'))
candlestick.update_xaxes(title_text = 'Time',
                         rangeslider_visible = True)

candlestick.update_layout(title = {'text': 'BTC Candelstick Chart','y':0.90,'x':0.5} , template="plotly_white",autosize=False,width=1500,height=500)
candlestick.update_yaxes(title_text = 'Price in USD')
candlestick.show()

<span id="Percentage-Value-Oscillator"><h2><center>Percentage Value Oscillator</center></h2></span>

$$ Fast \ exponential \ moving \ feature \ (FEMF) = Exponential \ moving \ feature \ of \ 'fast \ period' \ span $$

$$ Slow \ exponential \ moving \ feature \ (SEMF) = Exponential \ moving \ feature \ of \ 'slow \ period' \ span $$

$$ Raw \ Percentage \ Value \ Oscillator = \frac{SEMF - FEMF}{FEMF} $$

$$ Percentage \ Value \ Oscillator = Exponential \ moving \ feature \ of \ 'smoothing \ period' \ span \ over \ 'Raw \ Percentage \ Value \ Oscillator' $$

In [ ]:
from NitroFE import PercentageValueOscillator
ob=PercentageValueOscillator(fast_period=15, slow_period=10,smoothing_period=20)
res=ob.fit(btc['Close'][-500:],first_fit=True)[-100:]

In [ ]:
data=btc[-100:]
candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                           open = data[('Open')], 
                                           high = data[('High')], 
                                           low = data[('Low')], 
                                           close = data[('Close')])])
candlestick.update_xaxes(title_text = 'Time',
                         rangeslider_visible = True)

candlestick.update_layout(title = {'text': 'BTC Candelstick Chart','y':0.90,'x':0.5} , template="plotly_white",autosize=False,width=1200,height=500)
candlestick.update_yaxes(title_text = 'Price in USD')
candlestick.show()

featurefig=go.Figure(go.Scatter(x=data.index,y=[0]*len(data.index),mode='lines'))
featurefig.add_trace(go.Scatter(x=data.index, y=np.where(res.values>=0,res.values, 0),mode='lines', fill='tonexty', line_color='green'))
featurefig.add_trace(go.Scatter(x=data.index,y=[0]*len(data.index),mode='lines'))
featurefig.add_trace(go.Scatter(x=data.index, y=np.where(res.values<0,res.values,0),mode='lines',fill='tonexty',line_color='red'))

featurefig.update_layout(title = {'text': 'Percentage Value Oscillator','y':0.90,'x':0.5},autosize=False,width=1200,height=500 ,template="plotly_white")
featurefig.update_yaxes(title_text = 'Feature Value')
featurefig.show()

<span id="Moving-Average-Convergence-Divergence"><h2><center>Moving Average Convergence Divergence</center></h2></span>

$$ Moving \ average \ convergence \ divergence \ (MACD) = Exponential \ moving \ average \ of \ (Absolute \ Price \ Oscillator) $$

<br>

$$ Moving average \ convergence \ divergence \ histogram = (Absolute \ Price \ Oscillator)- (Exponential \ moving \ average \ of \ (Absolute \ Price)) $$

In [ ]:
from NitroFE import MovingAverageConvergenceDivergence
ob=MovingAverageConvergenceDivergence(fast_period=26, slow_period=12, smoothing_period=9)
res=ob.fit(btc['Close'][-500:],first_fit=True)[-100:]

In [ ]:
data=btc[-100:]
candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                           open = data[('Open')], 
                                           high = data[('High')], 
                                           low = data[('Low')], 
                                           close = data[('Close')])])
candlestick.update_xaxes(title_text = 'Time',
                         rangeslider_visible = True)

candlestick.update_layout(title = {'text': 'BTC Candelstick Chart','y':0.90,'x':0.5} , template="plotly_white",autosize=False,width=1200,height=500)
candlestick.update_yaxes(title_text = 'Price in USD')
candlestick.show()

featurefig=go.Figure(data = go.Bar(x=data.index, y=res.values,
                             marker_color=pd.Series(res.values).rolling(2,min_periods=2).agg(lambda x: 1 if x[-1:].values[0]>=x[-2:].values[0] else 0 ).\
                             fillna(1).replace(1,'green').replace(0,'red').values))
featurefig.update_layout(title = {'text': 'Moving Average Convergence Divergence Chart','y':0.90,'x':0.5},autosize=False,width=1200,height=500 ,template="plotly_white")
featurefig.update_yaxes(title_text = 'Feature Value')
featurefig.show()

<span id="Typical-Value"><h2><center>Typical Value</center></h2></span>

$$ Typical \ Value[t]= \frac{ \max{(dataframe[(t-lookback \ period) \to t])} +\min{(dataframe[(t-lookback \ period) \to t])} + dataframe[t] }{3} $$

In [ ]:
from NitroFE import TypicalValue
ob = TypicalValue(lookback_period=32)
res = ob.fit(btc['Close'][-500:],first_fit=True)[-100:]

In [ ]:
data=btc[-100:]
candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                           open = data[('Open')], 
                                           high = data[('High')], 
                                           low = data[('Low')], 
                                           close = data[('Close')])])

candlestick.add_trace(go.Scatter(x=data.index, y=res.values,mode='lines',line=dict(color='royalblue', width=5),name='Typical Value'))
candlestick.update_xaxes(title_text = 'Time',
                         rangeslider_visible = True)

candlestick.update_layout(title = {'text': 'BTC Candelstick Chart','y':0.90,'x':0.5} , template="plotly_white",autosize=False,width=1500,height=500)
candlestick.update_yaxes(title_text = 'Price in USD')
candlestick.show()

<span id="Kaufman-Efficiency"><h2><center>Kaufman Efficiency</center></h2></span>

$$ Kaufman \ Efficiency[t]= \frac{|dataframe[t]-dataframe[t-lookback \ period]|}{\sum_{i=t-lookback \ period+1}^{t}|dataframe[i]-dataframe[i-1]|} $$

In [ ]:
from NitroFE import KaufmanEfficiency
ob = KaufmanEfficiency(lookback_period=12)
res = ob.fit(btc['Close'][-500:], first_fit=True)[-100:]

In [ ]:
data=btc[-100:]
candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                           open = data[('Open')], 
                                           high = data[('High')], 
                                           low = data[('Low')], 
                                           close = data[('Close')])])
candlestick.update_xaxes(title_text = 'Time',
                         rangeslider_visible = True)

candlestick.update_layout(title = {'text': 'BTC Candelstick Chart','y':0.90,'x':0.5} , template="plotly_white",autosize=False,width=1200,height=500)
candlestick.update_yaxes(title_text = 'Price in USD')
candlestick.show()

featurefig=go.Figure(data = go.Scatter(x=data.index, y=res.values,mode='lines',line=dict(color='yellow', width=5)))
featurefig.update_layout(title = {'text': 'Kaufman Efficiency','y':0.90,'x':0.5},autosize=False,width=1200,height=500 ,template="plotly_white")
featurefig.update_yaxes(title_text = 'Feature Value')
featurefig.show()

<span id="Average-Directional-Movement-Index"><h2><center>Average Directional Movement Index</center></h2></span>

$$ Raw \ Positive \ Directional \ Movement[t] = \max{(dataframe[(t-directional \ movement \ lookback \ period ) \to t])} - \max{(dataframe[(t-2*directional\ movement \ lookback \ period) \to (t-directional \ movement \ lookback \ period)])} $$

<br>

$$ Raw \ Negative \ Directional \ Movement[t] = \min{(dataframe[(t-2*directional \ movement \ lookback \ period) \to (t-directional \ movement \ lookback \ period)])} - \min{(dataframe[(t-directional \ movement \ lookback \ period) \to t)} $$

<br>

$$ Positive \ Directional \ Movement[t] \ (+DM) = if (Raw \ Positive \ Directional_Movement[t] > Raw \ Negative \ Directional \ Movement[t]) \ and \ (Raw \ Positive \ Directional \ Movement[t] >0), then \ Raw \ Positive \ Directional \ Movement[t] \ else \ 0 $$

<br>

$$ Negative \ Directional \ Movement[t] \ (-DM) = if (Raw \ Negative \ Directional \ Movement[t] > Raw \ Positive \ Directional \ Movement[t]) \ and \ (Raw \ Negative \ Directional \ Movement[t] >0), then \ Raw \ Negative \ Directional \ Movement[t] \ else \ 0 $$

<br>

$$ Positive \ Directional \ Index \ (+DI) = 100*\frac{Smoothed \ moving \ average \ of \ Positive \ Directional \ Movement \ with \ length \ 'directional \ movement \ smoothing \ period' }{Average \ True \ Range \ of \ dataframe} $$

<br>

$$ Negative_Directional_Index \ (-DI) = 100*\frac{Smoothed \ moving \ average \ of \ Negative\ Directional \ Movement \ with \ length \ 'directional \ movement \ smoothing \ period' }{Average \ True \ Range \ of \ dataframe} $$

<br>

$$ average \ directional \ index \ (ADX) = Smoothed \ moving \ average \ of \ 100*\frac{+DI - -DI}{+DI + -DI} \ with \ length \ 'average \ directional \ movement \ smoothing \ period' $$

In [ ]:
from NitroFE import AverageDirectionalMovementIndex
ob = AverageDirectionalMovementIndex(
    directional_movement_lookback_period=8,
    directional_movement_smoothing_period=8,
    average_directional_movement_smoothing_period=8,
    directional_movement_smoothing_min_periods=0,
    true_range_lookback=8,
    average_true_range_span=8,
    average_directional_movement_min_periods=None)
res = ob.fit(btc['Close'][-500:], first_fit=True)['Close'][-100:]

In [ ]:
data=btc[-100:]
candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                           open = data[('Open')], 
                                           high = data[('High')], 
                                           low = data[('Low')], 
                                           close = data[('Close')])])
candlestick.update_xaxes(title_text = 'Time',
                         rangeslider_visible = True)

candlestick.update_layout(title = {'text': 'BTC Candelstick Chart','y':0.90,'x':0.5} , template="plotly_white",autosize=False,width=1200,height=500)
candlestick.update_yaxes(title_text = 'Price in USD')
candlestick.show()

featurefig=go.Figure(data = go.Bar(x=data.index, y=res.values,
                             marker_color=pd.Series(res.values).rolling(2,min_periods=2).agg(lambda x: 1 if x[-1:].values[0]>=x[-2:].values[0] else 0 ).\
                             fillna(1).replace(1,'green').replace(0,'red').values))
featurefig.update_layout(title = {'text': 'Average Directional Movement Index','y':0.90,'x':0.5},autosize=False,width=1200,height=500 ,template="plotly_white")
featurefig.update_yaxes(title_text = 'Feature Value')
featurefig.show()

<span id="Moving-average-Features"><h2><center>Moving average Features</center></h2></span>

<h4>Hull moving average</h4>

$$ \operatorname{WMA_(\frac{window}{2})} = Weighted \ Moving \ Average \ of \ length \ \frac{window}{2} \ over \ dataframe $$

$$ \operatorname{WMA_(window)} = Weighted \ Moving \ Average \ of \ length \ window \ over \ dataframe $$

$$ \operatorname{Raw \ Hull \ Moving \ Average} = 2*\operatorname{WMA_(\frac{window}{2})} - \operatorname{WMA_(window)} $$

$$ \operatorname{Hull \ Moving \ Average} = Weighted \ Moving \ Average \ of \ length \ \sqrt(window) over \operatorname{Raw \ Hull \ Moving \ Average} $$

<h4>Kaufman Adaptive Moving Average</h4>

$$ Current \ value[t] = dataframe[t] $$
<br>
$$ ER[t] = Kaufman \ Efficiency \ Ratio[t] $$
<br>
$$ fast \ ema \ alpha = \frac{2}{1+fast \ ema \ span} $$
<br>
$$ slow \ ema \ alpha = \frac{2}{1+slow \ ema \ span} $$
<br>
$$ Smoothing \ Constant[t] = (ER[t] * (fast \ ema \ alpha - slow \ ema \ alpha ) + slow \ ema \ alpha)^{2} $$
<br>
$$ KAMA[t]= KAMA[t-1] + Smoothing \ Constant[t] * ( Current \ value[t] - KAMA[t-1] ) $$

In [ ]:
from NitroFE import HullMovingFeature
ob = HullMovingFeature(window=20,operation=np.sum)
res = ob.fit(btc['Close'][-500:], first_fit=True)[-100:]

from NitroFE import KaufmanAdaptiveMovingAverage
ob = KaufmanAdaptiveMovingAverage(kaufman_efficiency_lookback_period=8,kaufman_efficiency_min_periods=0)
res2 = ob.fit(btc['Close'][-500:], first_fit=True)['Close'][-100:]

In [ ]:
data=btc[-100:]
candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                           open = data[('Open')], 
                                           high = data[('High')], 
                                           low = data[('Low')], 
                                           close = data[('Close')])])

candlestick.add_trace(go.Scatter(x=data.index, y=res.values,mode='lines',line=dict(color='royalblue', width=5),name='Hull Moving Average'))
candlestick.add_trace(go.Scatter(x=data.index, y=res2.values,mode='lines',line=dict(color='yellow', width=5),name='Kaufman Adaptive Moving Average'))
candlestick.update_xaxes(title_text = 'Time',
                         rangeslider_visible = True)

candlestick.update_layout(title = {'text': 'BTC Candelstick Chart','y':0.90,'x':0.5} , template="plotly_white",autosize=False,width=1500,height=500)
candlestick.update_yaxes(title_text = 'Price in USD')
candlestick.show()

<span id="Relative-Strength-Index"><h2><center>Relative Strength Index</center></h2></span>

$$ Difference \ value[t] (DFV) = dataframe[t] - dataframe[t-1] $$
<br>
$$ Upwards \ movement[t] (UM) = DFV[t] \ if \ DFV[t]>0 \ else \ 0 $$
<br>
$$ Downward \ movement[t] (DM) = |DFV[t]| \ if \ DFV[t]<0 \ else \ 0 $$
<br>
$$ RS = \frac{Smoothed \ moving \ average \ of \ UM \ over \ 'lookback \ period' \ period }{Smoothed \ moving \ average \ of \ DM \ over \ 'lookback \ period' \ period} $$
<br>
$$ RSI = 100 -\frac{100}{1+RS} $$

In [ ]:
from NitroFE import RelativeStrengthIndex
ob = RelativeStrengthIndex(lookback_period=8)
res = ob.fit(btc['Close'][-500:], first_fit=True)['Close'][-100:]

In [ ]:
data=btc[-100:]
candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                           open = data[('Open')], 
                                           high = data[('High')], 
                                           low = data[('Low')], 
                                           close = data[('Close')])])
candlestick.update_xaxes(title_text = 'Time',
                         rangeslider_visible = True)

candlestick.update_layout(title = {'text': 'BTC Candelstick Chart','y':0.90,'x':0.5} , template="plotly_white",autosize=False,width=1200,height=500)
candlestick.update_yaxes(title_text = 'Price in USD')
candlestick.show()

#featurefig=go.Figure(data = go.Scatter(x=data.index, y=res.values,mode='lines'))
#fig.add_trace(go.Scatter(x=data.index,y=[50]*len(data.index),mode='lines'))

featurefig=go.Figure(go.Scatter(x=data.index,y=[50]*len(data.index),mode='lines'))
featurefig.add_trace(go.Scatter(x=data.index, y=np.where(res.values>=50,res.values, 50),mode='lines', fill='tonexty', line_color='green'))
featurefig.add_trace(go.Scatter(x=data.index,y=[50]*len(data.index),mode='lines'))
featurefig.add_trace(go.Scatter(x=data.index, y=np.where(res.values<50,res.values,50),mode='lines',fill='tonexty',line_color='red'))

featurefig.update_layout(title = {'text': 'Relative Strength Index','y':0.90,'x':0.5},autosize=False,width=1200,height=500 ,template="plotly_white")
featurefig.update_yaxes(title_text = 'Feature Value')
featurefig.show()

<span id="Inverse-Fisher-Relative-Strength-Index"><h2><center>Inverse Fisher Relative Strength Index</center></h2></span>

$$ RSI = Relative \ Strength \ Index $$
<br>
$$ ranged \ RSI = 0.1 * (RSI - 50) $$
<br>
$$ wRSI = weighted \ moving \ average \ of ranged \ RSI \ of \ size \ 'lookback \ for \ inverse \ fisher' $$
<br>
$$ IFRSI = \frac{\exp(2wRSI)-1}{\exp(2wRSI)+1} $$

In [ ]:
from NitroFE import InverseFisherRelativeStrengthIndex
ob = InverseFisherRelativeStrengthIndex(lookback_period=8,lookback_for_inverse_fisher=8)
res = ob.fit(btc['Close'][-500:], first_fit=True)['Close'][-100:]

In [ ]:
data=btc[-100:]
candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                           open = data[('Open')], 
                                           high = data[('High')], 
                                           low = data[('Low')], 
                                           close = data[('Close')])])
candlestick.update_xaxes(title_text = 'Time',
                         rangeslider_visible = True)

candlestick.update_layout(title = {'text': 'BTC Candelstick Chart','y':0.90,'x':0.5} , template="plotly_white",autosize=False,width=1200,height=500)
candlestick.update_yaxes(title_text = 'Price in USD')
candlestick.show()

#featurefig=go.Figure(data = go.Scatter(x=data.index, y=res.values,line=dict(color='yellow', width=5)))

featurefig=go.Figure(go.Scatter(x=data.index,y=[0]*len(data.index),mode='lines'))
featurefig.add_trace(go.Scatter(x=data.index, y=np.where(res.values>=0,res.values, 0),mode='lines', fill='tonexty', line_color='green'))
featurefig.add_trace(go.Scatter(x=data.index,y=[0]*len(data.index),mode='lines'))
featurefig.add_trace(go.Scatter(x=data.index, y=np.where(res.values<0,res.values,0),mode='lines',fill='tonexty',line_color='red'))

featurefig.update_layout(title = {'text': 'Inverse Fisher Relative Strength Index','y':0.90,'x':0.5},autosize=False,width=1200,height=500 ,template="plotly_white")
featurefig.update_yaxes(title_text = 'Feature Value')
featurefig.show()

<span id="Aroon-Oscillator"><h2><center>Aroon Oscillator</center></h2></span>

$$ Aroon \ Up[t] = \frac{'lookback \ period'- Period \ since \ 'lookback \ period' \ High }{'lookback \ period'} $$
<br>
$$ Aroon \ Down[t] = \frac{'lookback \ period'- Period \ since \ 'lookback \ period' \ Low }{'lookback \ period'} $$
<br>
$$ Aroon \ Oscillator[t] = Aroon \ Up[t] - Aroon \ Down[t] $$

In [ ]:
from NitroFE import AroonOscillator
ob = AroonOscillator(lookback_period=8)
res = ob.fit(btc['Close'][-500:], first_fit=True)[-100:]

In [ ]:
data=btc[-100:]
candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                           open = data[('Open')], 
                                           high = data[('High')], 
                                           low = data[('Low')], 
                                           close = data[('Close')])])
candlestick.update_xaxes(title_text = 'Time',
                         rangeslider_visible = True)

candlestick.update_layout(title = {'text': 'BTC Candelstick Chart','y':0.90,'x':0.5} , template="plotly_white",autosize=False,width=1200,height=500)
candlestick.update_yaxes(title_text = 'Price in USD')
candlestick.show()

#featurefig=go.Figure(data = go.Scatter(x=data.index, y=res.values,line=dict(color='yellow', width=5)))

featurefig=go.Figure(go.Scatter(x=data.index,y=[0]*len(data.index),mode='lines'))
featurefig.add_trace(go.Scatter(x=data.index, y=np.where(res.values>=0,res.values, 0),mode='lines', fill='tonexty', line_color='green'))
featurefig.add_trace(go.Scatter(x=data.index,y=[0]*len(data.index),mode='lines'))
featurefig.add_trace(go.Scatter(x=data.index, y=np.where(res.values<0,res.values,0),mode='lines',fill='tonexty',line_color='red'))


featurefig.update_layout(title = {'text': 'Aroon Oscillator','y':0.90,'x':0.5},autosize=False,width=1200,height=500 ,template="plotly_white")
featurefig.update_yaxes(title_text = 'Feature Value')
featurefig.show()

<span id="Triple-Exponential-Moving-Average-Oscillator"><h2><center>Triple Exponential Moving Average Oscillator</center></h2></span>

$$ Triple \ Exponential \ Moving \ Average \ Oscillator[t] = \frac{Triple \ Exponential \ Moving \ Average[t]-Triple \ Exponential \ Moving \ Average[t-1]}{Triple \ Exponential \ Moving \ Average[t-1]} $$

In [ ]:
from NitroFE import TripleExponentialMovingAverageOscillator
ob = TripleExponentialMovingAverageOscillator(span=8)
res = ob.fit(btc['Close'][-500:], first_fit=True)[-100:]

In [ ]:
data=btc[-100:]
candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                           open = data[('Open')], 
                                           high = data[('High')], 
                                           low = data[('Low')], 
                                           close = data[('Close')])])
candlestick.update_xaxes(title_text = 'Time',
                         rangeslider_visible = True)

candlestick.update_layout(title = {'text': 'BTC Candelstick Chart','y':0.90,'x':0.5} , template="plotly_white",autosize=False,width=1200,height=500)
candlestick.update_yaxes(title_text = 'Price in USD')
candlestick.show()

#featurefig=go.Figure(data = go.Scatter(x=data.index, y=res.values,line=dict(color='yellow', width=5)))

featurefig=go.Figure(go.Scatter(x=data.index,y=[0]*len(data.index),mode='lines'))
featurefig.add_trace(go.Scatter(x=data.index, y=np.where(res.values>=0,res.values, 0),mode='lines', fill='tonexty', line_color='green'))
featurefig.add_trace(go.Scatter(x=data.index,y=[0]*len(data.index),mode='lines'))
featurefig.add_trace(go.Scatter(x=data.index, y=np.where(res.values<0,res.values,0),mode='lines',fill='tonexty',line_color='red'))

featurefig.update_layout(title = {'text': 'Triple Exponential Moving Average Oscillator','y':0.90,'x':0.5},autosize=False,width=1200,height=500 ,template="plotly_white")
featurefig.update_yaxes(title_text = 'Feature Value')
featurefig.show()

<span id="Combination-Features"><h1><center>Combination Features</center></h1></span>

Here are a few examples of the complex features you can make with the combination of the basic set of features !

<span id="Typical-Value-Moving-Average-Convergence-Divergence"><h2><center>Typical Value Moving Average Convergence Divergence</center></h2></span>

In [ ]:
from NitroFE import TypicalValue
ob = TypicalValue(lookback_period=32)
res = ob.fit(btc['Close'][-500:],first_fit=True)


from NitroFE import MovingAverageConvergenceDivergence
ob=MovingAverageConvergenceDivergence(fast_period=26, slow_period=12, smoothing_period=9)
res=ob.fit(res,first_fit=True)[-100:]

In [ ]:
data=btc[-100:]
candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                           open = data[('Open')], 
                                           high = data[('High')], 
                                           low = data[('Low')], 
                                           close = data[('Close')])])
candlestick.update_xaxes(title_text = 'Time',
                         rangeslider_visible = True)

candlestick.update_layout(title = {'text': 'BTC Candelstick Chart','y':0.90,'x':0.5} , template="plotly_white",autosize=False,width=1200,height=500)
candlestick.update_yaxes(title_text = 'Price in USD')
candlestick.show()

featurefig=go.Figure(data = go.Bar(x=data.index, y=res.values,
                             marker_color=pd.Series(res.values).rolling(2,min_periods=2).agg(lambda x: 1 if x[-1:].values[0]>=x[-2:].values[0] else 0 ).\
                             fillna(1).replace(1,'#1f77b4').replace(0,'#9467bd').values))
featurefig.update_layout(title = {'text': 'Typical Value Moving Average Convergence Divergence Chart','y':0.90,'x':0.5},autosize=False,width=1200,height=500 ,template="plotly_white")
featurefig.update_yaxes(title_text = 'Feature Value')
featurefig.show()

<span id="Typical-Value-Volume-Weighted-Moving-Average"><h2><center>Typical Value Volume Weighted Moving Average</center></h2></span>

In [ ]:
from NitroFE import TypicalValue
ob = TypicalValue(lookback_period=32)
res = ob.fit(btc['Close'][-500:],first_fit=True)

from NitroFE import SeriesWeightedMovingFeature
ob = SeriesWeightedMovingFeature(lookback_period=8, operation= np.mean)
res = ob.fit(res,btc['Volume'][-500:],first_fit=True)['Close'][-100:]

from NitroFE import ExponentialMovingFeature
ob = ExponentialMovingFeature(span=8)
res2 = ob.fit(btc['Close'][-500:], first_fit=True)[-100:]

In [ ]:
data=btc[-100:]
candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                           open = data[('Open')], 
                                           high = data[('High')], 
                                           low = data[('Low')], 
                                           close = data[('Close')])])

candlestick.add_trace(go.Scatter(x=data.index, y=res.values,mode='lines',line=dict(color='#1f77b4', width=5),name='Typical Value Volume Weighted Moving Average'))
candlestick.add_trace(go.Scatter(x=data.index, y=res2.values,mode='lines',line=dict(color='#9467bd', width=5),name='Exponential Moving Feature'))
candlestick.update_xaxes(title_text = 'Time',
                         rangeslider_visible = True)

candlestick.update_layout(title = {'text': 'BTC Candelstick Chart','y':0.90,'x':0.5} , template="plotly_white",autosize=False,width=1500,height=500)
candlestick.update_yaxes(title_text = 'Price in USD')
candlestick.show()

<span id="Volume-Moving-Average-Convergence-Divergence"><h2><center>Volume Moving Average Convergence Divergence</center></h2></span>

In [ ]:
from NitroFE import TypicalValue
ob = TypicalValue(lookback_period=32)
res = ob.fit(btc['Volume'][-500:],first_fit=True)


from NitroFE import MovingAverageConvergenceDivergence
ob=MovingAverageConvergenceDivergence(fast_period=26, slow_period=12, smoothing_period=9)
res=ob.fit(res,first_fit=True)[-100:]

In [ ]:
data=btc[-100:]
candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                           open = data[('Open')], 
                                           high = data[('High')], 
                                           low = data[('Low')], 
                                           close = data[('Close')])])
candlestick.update_xaxes(title_text = 'Time',
                         rangeslider_visible = True)

candlestick.update_layout(title = {'text': 'BTC Candelstick Chart','y':0.90,'x':0.5} , template="plotly_white",autosize=False,width=1200,height=500)
candlestick.update_yaxes(title_text = 'Price in USD')
candlestick.show()

featurefig=go.Figure(data = go.Bar(x=data.index, y=res.values,
                             marker_color=pd.Series(res.values).rolling(2,min_periods=2).agg(lambda x: 1 if x[-1:].values[0]>=x[-2:].values[0] else 0 ).\
                             fillna(1).replace(1,'#1f77b4').replace(0,'#9467bd').values))
featurefig.update_layout(title = {'text': 'Volume Moving Average Convergence Divergence','y':0.90,'x':0.5},autosize=False,width=1200,height=500 ,template="plotly_white")
featurefig.update_yaxes(title_text = 'Feature Value')
featurefig.show()

<h2><center>Do check out the feature selection library "Zoofs" as well !</center></h2>

📚 To read more about Zoofs you can visit the documentation page https://jaswinder9051998.github.io/zoofs/.

You can find examples for feature selection using all six available algorithms.


🌟 Like this Project? Give us a star on GitHub